# 데이터 로드 및 결측치 확인

In [2]:
import pandas as pd

df = pd.read_excel("sp500_fill_in_df_2.xlsx")
df = df.iloc[:, 1:]

In [3]:
df

,shortName,symbol,country,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,...,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
0,"Agilent Technologies, Inc.",A,United States,Diagnostics & Research,Healthcare,18400,10,4,5,9,...,1175000064,2891000064,1.546,2.370,7040000000,0.10401,0.068,0.54588,0.29247,0.25114
1,"American Airlines Group, Inc.",AAL,United States,Airlines,Industrials,130800,7,4,6,9,...,12316000256,42929000448,0.582,0.718,52260999168,0.03864,0.370,0.26094,0.12323,0.07937
2,Advance Auto Parts Inc.,AAP,United States,Specialty Retail,Consumer Cyclical,40000,3,5,6,2,...,226499008,4636292096,0.202,1.244,11198105600,0.03311,0.013,0.44015,0.08317,0.05720
3,Apple Inc.,AAPL,United States,Consumer Electronics,Technology,164000,4,1,5,1,...,55872000000,109614997504,0.764,0.940,385095008256,0.20559,-0.025,0.43181,0.32145,0.29163
4,AbbVie Inc.,ABBV,United States,Drug Manufacturers—General,Healthcare,50000,9,7,7,9,...,6721999872,62461001728,0.659,0.960,56740999168,0.09443,-0.097,0.70959,0.52031,0.36979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,"Yum! Brands, Inc.",YUM,United States,Restaurants,Consumer Cyclical,36000,1,3,5,7,...,373000000,12490999808,0.640,0.994,6940000256,0.23843,0.063,0.48501,0.33775,0.31787
486,"Zimmer Biomet Holdings, Inc.",ZBH,United States,Medical Devices,Healthcare,18000,2,8,10,4,...,330200000,5980400128,0.733,1.889,7107700224,0.04118,0.101,0.71576,0.32870,0.19864
487,Zebra Technologies Corporation,ZBRA,United States,Communication Equipment,Technology,10500,1,9,8,9,...,109000000,2288000000,0.422,0.877,5753999872,0.08776,-0.019,0.46124,0.20003,0.16597
488,Zions Bancorporation N.A.,ZION,United States,Banks—Regional,Financial Services,10064,4,7,7,6,...,4402999808,13338999808,1.200,1.900,3104999936,0.01010,0.104,0.00000,0.00000,0.39581


In [4]:
# 결측치 확인하고 결측치 수 순서대로 정렬
missing_values = df.isnull().sum()
missing_values.sort_values(ascending=False, inplace=True)

# 결측치 있는 열 확인
missing_values[missing_values > 0]

Series([], dtype: int64)

# Pipeline을 이용한 데이터 전처리

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 수치형
num_features = ['marketCap', 'dividendYield', 'payoutRatio', 'beta', 'trailingPE', 'averageVolume', 'profitMargins', 
                'heldPercentInsiders', 'heldPercentInstitutions', 'bookValue', 'priceToBook', 'pegRatio', 
                'enterpriseToEbitda', '52WeekChange', 'recommendationMean', 'numberOfAnalystOpinions', 'totalCash', 
                'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'returnOnAssets', 'revenueGrowth', 
                'grossMargins', 'ebitdaMargins', 'operatingMargins']
# 범주형
cat_features = ['industry', 'sector']

# transformer 생성
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# preprocessor 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

# df_preprocessed에 결과 저장
df_preprocessed = preprocessor.fit_transform(df)

df_preprocessed.shape

(490, 149)

# knn 모델 훈련

In [7]:
from sklearn.neighbors import NearestNeighbors

# k-NN 모델 훈련
knn = NearestNeighbors(n_neighbors=6, metric='cosine')
knn.fit(df_preprocessed)

NearestNeighbors(metric='cosine', n_neighbors=6)

# 주어진 종목 코드와 유사한 상위 5개 종목 출력

In [8]:
def recommend_stocks(symbol):
    # 주어진 symbol에 해당하는 주식의 index를 반환
    stock_index = df[df['symbol'] == symbol].index[0]

    # 주어진 stock의 knn 모델에 대한 최근접 이웃 indices를 호출
    neighbor_indices = knn.kneighbors(df_preprocessed[stock_index].reshape(1, -1), return_distance=False)
    #reshape(1,-1)은 knn이 2차원 배열을 받기 때문에 사용

    # 최근접 이웃을 호출
    neighbor_symbols = df.iloc[neighbor_indices[0]]

    # input 주식은 제외하고 추천
    recommendations = neighbor_symbols[neighbor_symbols['symbol'] != symbol]

    return recommendations

recommend_stocks("AAPL")

,shortName,symbol,country,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,...,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
314,Microsoft Corporation,MSFT,United States,Software—Infrastructure,Technology,221000,6,3,3,2,...,104419000320,79312003072,1.655,1.913,207590998016,0.14829,0.071,0.68522,0.48210,0.41415
205,Alphabet Inc.,GOOGL,United States,Internet Content & Information,Communication Services,190711,8,2,10,10,...,115101999104,29493000192,2.195,2.353,284612001792,0.12415,0.026,0.55302,0.30742,0.25355
204,Alphabet Inc.,GOOG,United States,Internet Content & Information,Communication Services,190711,8,2,10,10,...,115101999104,29493000192,2.195,2.353,284612001792,0.12415,0.026,0.55302,0.30742,0.25355
34,"Amazon.com, Inc.",AMZN,United States,Internet Retail,Consumer Cyclical,1541000,5,10,10,2,...,64405000192,178546999296,0.689,0.923,524896993280,0.01906,0.094,0.44728,0.10896,0.02542
295,"Meta Platforms, Inc.",META,United States,Internet Content & Information,Communication Services,77114,8,9,10,10,...,37439000576,28262000640,1.910,2.068,117346000896,0.11973,0.026,0.79576,0.36180,0.28465


# 포트폴리오 기반 추천 시스템 구현

In [11]:
from sklearn.neighbors import NearestNeighbors

# k-NN 모델 훈련 (전체 종목의 순서를 출력)
knn_2 = NearestNeighbors(n_neighbors=490, metric='cosine')
knn_2.fit(df_preprocessed)

NearestNeighbors(metric='cosine', n_neighbors=490)

In [16]:
def knn_df(symbol):
    stock_index = df[df['symbol'] == symbol].index[0]
    
    #distance와 indices dataframe을 만들고 concat으로 병합
    df_distance = pd.DataFrame(knn_2.kneighbors(df_preprocessed[stock_index].reshape(1, -1), return_distance=True)[0]).T
    df_indices = pd.DataFrame(knn_2.kneighbors(df_preprocessed[stock_index].reshape(1, -1), return_distance=True)[1]).T
    df_concat = pd.concat([df_distance, df_indices], axis = 1)
    df_concat.columns = ['distance', 'index']
    
    return df_concat.sort_values(by = 'index').reset_index().iloc[:, 1:2]

In [24]:
import numpy as np

def recommended_stocks_portfolio(portfolio):
    #주어진 종목 별 가중치 계산
    total = [portfolio[stock] for stock in portfolio]
    
    for stock in portfolio:
        portfolio[stock] = portfolio[stock] / np.sum(total)
    
    #portfolio_list에 knn_df를 계산하고 concat하기
    portfolio_list = list()
    
    for stock in portfolio:
        knn_df_stock = knn_df(stock) * portfolio[stock]
        portfolio_list.append(knn_df_stock)
        
    portfolio_df = pd.concat(portfolio_list, axis = 1)
    
    #total이라는 열을 만들어서 가중합 계산
    portfolio_df['total'] = portfolio_df.sum(axis=1)
    
    #가중합이 낮은 순으로 정렬
    portfolio_df_sorted = portfolio_df.sort_values(by = 'total')
    
    #symbol_list에 포트폴리오 내부 종목 티커를 작성
    symbol_list = [stock for stock in portfolio]
    
    #return_index에 portfolio 내부 종목 인덱스 정보 반환
    return_index = df[df['symbol'].isin(symbol_list)].index
    
    #정렬된 df에서 return_index에 해당하는 종목들을 drop
    portfolio_df_sorted = portfolio_df_sorted.drop(return_index)
    
    #index 기준으로 return_df를 인덱싱하여 반환. 
    return df.iloc[portfolio_df_sorted.index, :]

In [32]:
recommended_stocks_portfolio({"AAPL": 500, "TSLA": 500, "SBUX": 1000}).iloc[:5, ]

,shortName,symbol,country,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,...,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
34,"Amazon.com, Inc.",AMZN,United States,Internet Retail,Consumer Cyclical,1541000,5,10,10,2,...,64405000192,178546999296,0.689,0.923,524896993280,0.01906,0.094,0.44728,0.10896,0.02542
205,Alphabet Inc.,GOOGL,United States,Internet Content & Information,Communication Services,190711,8,2,10,10,...,115101999104,29493000192,2.195,2.353,284612001792,0.12415,0.026,0.55302,0.30742,0.25355
295,"Meta Platforms, Inc.",META,United States,Internet Content & Information,Communication Services,77114,8,9,10,10,...,37439000576,28262000640,1.910,2.068,117346000896,0.11973,0.026,0.79576,0.36180,0.28465
277,"Lowe's Companies, Inc.",LOW,United States,Home Improvement Retail,Consumer Cyclical,300000,5,7,3,3,...,3372999936,40597000192,0.176,1.245,95746998272,0.15953,-0.055,0.33144,0.14777,0.12748
324,"Nike, Inc.",NKE,United States,Footwear & Accessories,Consumer Cyclical,79100,8,10,2,7,...,10675000320,12144000000,1.600,2.723,51216998400,0.09497,0.048,0.43525,0.13171,0.11549


# User-Interactive한 포트폴리오 빌더 구현

In [33]:
def print_result(return_df, num):
    for i in range(num):
        ith_df = return_df.iloc[i, :]
        print(f"""{i+1}:  {ith_df['shortName']} ({ith_df['symbol']}).
    Industry / Sector: {ith_df['industry']} / {ith_df['sector']}
    Dividend Yield: {ith_df['dividendYield']}, Payout Ratio: {ith_df['payoutRatio']}
    Trailing PER: {ith_df['trailingPE']}, PBR: {ith_df['priceToBook']}, Profit Margins: {ith_df['profitMargins']}
    Overall Risk: {ith_df['overallRisk']}, beta: {ith_df['beta']}""")

In [34]:
def build_portfolio(initial_stock, num_stocks=20):
    # portfolio_list에 선정 종목 입력 
    portfolio_list = [initial_stock]
    
    # hide_list에 감출 종목 입력
    hide_list = list()
    
    #가중치는 다 1로 초기화
    value = 1

    # num_stocks를 만족할 때까지 반복
    while len(portfolio_list) < num_stocks:
        
        #전 종목의 가중치는 1로 초기화
        portfolio_dict = {key: value for key in portfolio_list}
        
        # 현재 포트폴리오를 기반으로 종목 추천
        recommendations = recommended_stocks_portfolio(portfolio_dict)
        
        # 감출 종목은 감추기
        recommendations = recommendations[~recommendations['symbol'].isin(hide_list)]

        # 유저에게 추천 포트폴리오 출력
        print()
        print_result(recommendations, 5)
        print(f"\n현재 포트폴리오: {portfolio_list}")
        
        # 유저에게 선택지 제공
        selected_option = input("\n'추가'를 입력해 포트폴리오에 종목 추가\n'제거'를 입력해 포트폴리오에 종목 제거\n'감추기'를 입력해 종목 추천 감추기: ")
        
        if selected_option == '추가':
            selected_stock = input("\n추가할 종목의 티커명을 입력해주세요: ")
            # 선택된 주식을 포트폴리오에 추가
            portfolio_list.append(selected_stock)
            
        elif selected_option == '제거':
            selected_stock = input("\n제거할 종목의 티커명을 입력해주세요: ")
            # 선택된 주식을 포트폴리오에서 제거
            portfolio_list.remove(selected_stock)
            
        elif selected_option == '감추기':
            selected_stock = input("\n감출 종목의 티커명을 입력해주세요: ")
            hide_list.append(selected_stock)

    return portfolio_list

In [35]:
build_portfolio("AAPL", 5)


1:  Microsoft Corporation (MSFT).
    Industry / Sector: Software—Infrastructure / Technology
    Dividend Yield: 0.0076, Payout Ratio: 0.28170002
    Trailing PER: 39.941505, PBR: 13.564061, Profit Margins: 0.33248
    Overall Risk: 2, beta: 0.91086
2:  Alphabet Inc. (GOOGL).
    Industry / Sector: Internet Content & Information / Communication Services
    Dividend Yield: 0.0, Payout Ratio: 0.0
    Trailing PER: 27.117777, PBR: 5.950651, Profit Margins: 0.20584999
    Overall Risk: 10, beta: 1.052595
3:  Alphabet Inc. (GOOG).
    Industry / Sector: Internet Content & Information / Communication Services
    Dividend Yield: 0.0, Payout Ratio: 0.0
    Trailing PER: 27.40625, PBR: 5.9872236, Profit Margins: 0.20584999
    Overall Risk: 10, beta: 1.052595
4:  Amazon.com, Inc. (AMZN).
    Industry / Sector: Internet Retail / Consumer Cyclical
    Dividend Yield: 0.0, Payout Ratio: 0.0
    Trailing PER: 330.14633, PBR: 8.9856615, Profit Margins: 0.00818
    Overall Risk: 10, beta: 1.25785


1:  Amazon.com, Inc. (AMZN).
    Industry / Sector: Internet Retail / Consumer Cyclical
    Dividend Yield: 0.0, Payout Ratio: 0.0
    Trailing PER: 330.14633, PBR: 8.9856615, Profit Margins: 0.00818
    Overall Risk: 10, beta: 1.257855
2:  NVIDIA Corporation (NVDA).
    Industry / Sector: Semiconductors / Technology
    Dividend Yield: 0.0004, Payout Ratio: 0.0833
    Trailing PER: 246.47644, PBR: 47.480583, Profit Margins: 0.18521
    Overall Risk: 8, beta: 1.750683
3:  Microsoft Corporation (MSFT).
    Industry / Sector: Software—Infrastructure / Technology
    Dividend Yield: 0.0076, Payout Ratio: 0.28170002
    Trailing PER: 39.941505, PBR: 13.564061, Profit Margins: 0.33248
    Overall Risk: 2, beta: 0.91086
4:  Alphabet Inc. (GOOG).
    Industry / Sector: Internet Content & Information / Communication Services
    Dividend Yield: 0.0, Payout Ratio: 0.0
    Trailing PER: 27.40625, PBR: 5.9872236, Profit Margins: 0.20584999
    Overall Risk: 10, beta: 1.052595
5:  Intel Corporati

['AAPL', 'TSLA', 'META', 'AMD', 'INTC']